In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as soup
import pandas as pd



# this will collect the url for every year page and store it in year_links
first_url = 'https://www.foxsports.com/nfl/stats?season=2019&week=101&category=RUSHING'
searching_url = first_url
url_client = urlopen(searching_url)
html_url = url_client.read()
url_client.close()
soup_url = soup(html_url, 'html.parser')

page_stats = soup_url.table.findAll('th')
page_stats.pop(0)
rush_stats = ['Player Name', 'Team']
for stat in page_stats:
    rush_stats.append(stat.text.strip())

rush_stats
numbers = soup_url.table.tbody.findAll('td', {'class' : 'wisbb_priorityColumn'})



names = soup_url.table.tbody.findAll('a')
name_list_raw = []
name_list = []
count = 0
for name in names:   
    name_list_raw.append(names[count].span)
    count += 1


'Mack, Marlon'

[<span>Mack, Marlon</span>,
 None,
 <span>McCaffrey, Christian</span>,
 None,
 <span>Barkley, Saquon</span>,
 None,
 <span>Cook, Dalvin</span>,
 None,
 <span>Ingram II, Mark</span>,
 None,
 <span>Gurley II, Todd</span>,
 None,
 <span>Kamara, Alvin</span>,
 None,
 <span>Jacobs, Josh</span>,
 None,
 <span>Henry, Derrick</span>,
 None,
 <span>Hyde, Carlos</span>,
 None,
 <span>Johnson, David</span>,
 None,
 <span>McCoy, LeSean</span>,
 None,
 <span>Chubb, Nick</span>,
 None,
 <span>Jones II, Ronald</span>,
 None,
 <span>Singletary, Devin</span>,
 None,
 <span>Fournette, Leonard</span>,
 None,
 <span>Levine Sr., Anthony</span>,
 None,
 <span>Bell, Le'Veon</span>,
 None,
 <span>Ekeler, Austin</span>,
 None,
 <span>Johnson Jr., Duke</span>,
 None,
 <span>Jackson, Justin</span>,
 None,
 <span>Freeman, Royce</span>,
 None,
 <span>Edwards, Gus</span>,
 None,
 <span>Brown, Malcolm</span>,
 None,
 <span>Elliott, Ezekiel</span>,
 None,
 <span>Johnson, Kerryon</span>,
 None,
 <span>Mattison, Alexan